# JETANK 舵机控制

JETANK是一款基于jetbot开发的开源机器人产品，它在jetbot的基础上增加了5个舵机和一对履带，这让它拥有了可以俯仰的摄像头以及可以夹取物体的机械臂。

在控制机械臂之前，确保舵机已经连接好，并且已经按照教程内容正确校准舵机中位。当舵机连接正确后上电，转动舵机会有明显的阻尼感。

在接下来的文档中，我们将使用Python代码来控制舵机。


### 导入舵机控制函数库

要控制舵机，首先我们需要导入用来控制舵机的库，运行这句代码前请注意，由于Jetson Nano系统本身有服务已经占用了ttyTHS1(Jetson Nano与舵机通信的串口)，如果我们的安装脚本执行一切正常，会关闭掉之前占用ttyTHS1的服务，脚本也会自动添加赋予ttyTHS1串口0666的权限。

如果之前的安装脚本执行失败，在这里使用ttyTHS1时可能会遇到Permission Denied的报错，如果遇到这种报错，以下是解决方法：
- 点击JupyterLab左上角的加号，会打开一个新的Launcher选项卡，在此选项卡中的Terminal打开Jetson Nano的控制台。
- 在控制台中输入以下内容后按回车
    > sudo chmod 666 /dev/ttyTHS1
- 输入Jetpack的默认密码`jetbot`，按回车确认修改权限。
- 在左侧的`JETANK_1_servos.ipynb`上点击右键，点击`Shut Down Kernal`。
- 关闭当前的`JETANK_1_servos.ipynb`选项卡后再重新双击左侧的`JETANK_1_servos.ipynb`运行新的Kernal。

然后可以选择以下代码后按Ctrl+Enter来导入用于控制舵机的库，导入成功后会有如下提示：
Succeeded to open the port
Succeeded to change the baudrate


In [1]:
from SCSCtrl import TTLServo

Succeeded to open the port
Succeeded to change the baudrate


接下来我们使用最简单的舵机控制函数`servoAngleCtrl()`来控制舵机，这个函数可以用于控制某一个舵机转动，调用这个函数时需要四个参数：
`servoAngleCtrl(servoID, angleInput, direction, speed)`

> **servoID:** 被控制舵机的编号，其中负责机械臂（摄像头）水平方向（Pan）转动的舵机为1号；负责机械臂根部俯仰运动的舵机为2号舵机；负责机械臂中间关节俯仰运动的舵机为3号舵机；负责机械臂夹头夹取物体的舵机为4号；负责摄像头俯仰（Tilt）运动的舵机为5号舵机。

> **angleInput:** 控制舵机的转动角度，理论上范围可以为+-150，实际上考虑到机械结构干涉，转动范围会受限。此处应当注意的是，angleInput的角度是相对于舵机中位的角度，如果你希望舵机转动到中位，让angleInput的值为0即可。

> **direction:** 这个参数的值可以为1或-1，这个参数用于调整舵机的转动方向，例如`TTLServo.servoAngleCtrl(4, -40, 1, 150)`和`TTLServo.servoAngleCtrl(4, 40, -1, 150)`都是控制舵机转动到同一个位置。

> **speed:** 这个参数用于控制舵机的转动速度，当这个数值越大转动速度越快，范围是1-1500，但是需要注意的是，当这个参数为0时，也代表最大的转动速度。

运行下面的代码cell，控制1、2、3、4号舵机以150的速度转动到中位。

In [2]:
TTLServo.servoAngleCtrl(1, 0, 1, 150)
TTLServo.servoAngleCtrl(2, 0, 1, 150)
TTLServo.servoAngleCtrl(3, 0, 1, 150)
TTLServo.servoAngleCtrl(4, 0, 1, 150)
TTLServo.servoAngleCtrl(5, 0, 1, 150)

512

### 逆运动学控制

由于机械臂夹头的位置由两个舵机耦合控制，如果我们分别控制两个舵机的转动会很难控制机械臂夹头的位置，所以这里使用了逆运动学函数来计算与夹头位置耦合的两个舵机的转动角度，只需要输入你希望夹头所在位置的相对坐标值`（x, y）`即可控制机械臂的运动。

运行下面的代码控制机械臂夹头运动到`（100，0）`的位置，100是X值，X值越大夹头位置越靠前；0是Y值，Y值越大位置越高。单位为毫米。

In [3]:
TTLServo.xyInput(100, 0)

[-114.6243183521641, 55.377838160268176]

为了更加直观地表现XY值的关系，这里让机械臂从`(100, 0)`步进运动到`(200, 0)`，然后再回到位置`(150, 0)`。
这里的运动仅改变位置的X参数。
运行下面的代码，同时观察机械臂的运动。

In [4]:
import time

# 先让舵机运动到（100，0）
TTLServo.xyInput(100, 0)
time.sleep(1)

# 从（100，0）到（200，0）步进运动
for i in range(100, 201):
    TTLServo.xyInput(i, 0)
    time.sleep(0.01)
time.sleep(1)

# 在（100，0）与（100，100）之间做平滑往返运动，第三个参数为运动到该位置所需要的时间，单位秒。
for i in range(0,4):
    TTLServo.xyInputSmooth(100, 0, 1)
    time.sleep(2)
    TTLServo.xyInputSmooth(100, 100, 1)
    time.sleep(2)

# 控制舵机运动到（150，0）
TTLServo.xyInput(150, 0)
time.sleep(1)

上面的代码只是X轴运动，下面的代码仅有Y值变化，运行下面的代码，同时观察机械臂的运动。

In [5]:
# 先让舵机运动到（150，-50）
TTLServo.xyInput(150, -50)
time.sleep(1)

# 从（150，-50）到（150，50）步进运动
for i in range(-50, 51):
    TTLServo.xyInput(150, i)
    time.sleep(0.01)
time.sleep(1)

# 控制舵机运动到（150， 0）
TTLServo.xyInput(150, 0)
time.sleep(1)


在实际的使用过程中还有一种情况，例如摄像头和机械臂的水平方向（Pan）运动，发出一个指令让其开始旋转，但并不确定具体的旋转角度，而是开始转动后随时有可能需要控制它停下来。类似这种情况也可以使用`servoAngleCtrl(1, 80, 1, 150)`，可以用来控制摄像头开始向右旋转，如果把80改为-80则是开始向左旋转，这里的80和-80不仅是用来设置方向的，也可以用来设置限位角度，当舵机开始转动后，任意时刻调用`servoStop(servoID)`即可让处在运动状态的舵机停下来，其中`servoID`参数是需要停止的舵机编号。

In [6]:
TTLServo.servoAngleCtrl(1, 80, 1, 150)
time.sleep(2)
TTLServo.servoStop(1)

以上就是舵机的初级使用方法。